# 概要

すべてのレイヤを直列に（シーケンシャルに）接続するモデルはSequential APIで簡単に実現できる。<br>
一方それ以上に複雑なモデルは対応できないため関数型APIを使ってニューラルネットワークを構築する。

# ワイド・アンド・ディープニューラルネットワーク

シーケンシャルなニューラルネットワークではすべてのデータに全層の強制する。<br>
そのためデータに含まれる単純なパターンが層を通過する中で歪められてしまうことがある。<br>
そこで層の通過をスキップする"ワイド"パスを含むモデルを作成する。

今回作成するモデルは以下：

![ワイドアンドディープニューラルネット](./fig/wide_and_deep_nural_net.png)

## パッケージインポート

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd

print(f'tensorflow ver.{tf.__version__}')
print(f'keras ver.{keras.__version__}')

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

tensorflow ver.2.8.0
keras ver.2.8.0


## データロードと前処理

###  データロード（sklearn.datasets.fetch_california_housing）

In [2]:
housing = fetch_california_housing()

x_train_full, x_test, y_train_full, y_test = train_test_split(housing.data, housing.target)

### validation分割

In [3]:
x_train, x_valid, y_train, y_valid = train_test_split(x_train_full, y_train_full)

# データサイズを確認
print(f'x_train.shape : {x_train.shape}')
print(f'y_train.shape : {y_train.shape}')
print(f'x_valid.shape : {x_valid.shape}')
print(f'y_valid.shape : {y_valid.shape}')

x_train.shape : (11610, 8)
y_train.shape : (11610,)
x_valid.shape : (3870, 8)
y_valid.shape : (3870,)


trainデータを確認

In [4]:
pd_x_train = pd.DataFrame(x_train, columns=housing.feature_names)
display(pd_x_train)
pd_x_train.info()
pd_x_train.describe()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,3.5250,34.0,4.296296,0.895062,398.0,2.456790,37.63,-122.49
1,5.4063,36.0,5.378652,1.011236,2039.0,2.291011,37.41,-122.13
2,3.5129,34.0,5.234072,1.060942,2356.0,3.263158,38.01,-121.90
3,4.1500,46.0,4.039216,0.862745,134.0,2.627451,37.42,-122.14
4,2.3547,31.0,4.174497,1.040268,821.0,1.836689,37.33,-121.95
...,...,...,...,...,...,...,...,...
11605,3.5972,26.0,5.000000,1.097902,1150.0,2.680653,33.90,-118.14
11606,3.5444,12.0,4.786854,1.027230,2029.0,1.905164,38.68,-121.29
11607,10.2849,26.0,7.807095,1.008869,1336.0,2.962306,33.64,-117.87
11608,4.5100,9.0,5.736216,1.073514,2630.0,2.843243,34.08,-117.16


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11610 entries, 0 to 11609
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   MedInc      11610 non-null  float64
 1   HouseAge    11610 non-null  float64
 2   AveRooms    11610 non-null  float64
 3   AveBedrms   11610 non-null  float64
 4   Population  11610 non-null  float64
 5   AveOccup    11610 non-null  float64
 6   Latitude    11610 non-null  float64
 7   Longitude   11610 non-null  float64
dtypes: float64(8)
memory usage: 725.8 KB


,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
count,11610.000000,11610.000000,11610.000000,11610.000000,11610.000000,11610.000000,11610.000000,11610.000000
mean,3.858186,28.573643,5.444476,1.100467,1430.212748,3.007000,35.634738,-119.563834
std,1.875128,12.596228,2.448460,0.491478,1162.821538,5.216354,2.133309,1.992910
min,0.499900,1.000000,0.846154,0.375000,3.000000,0.692308,32.540000,-124.300000
25%,2.564300,18.000000,4.446701,1.005894,788.000000,2.436990,33.930000,-121.780000
50%,3.543250,29.000000,5.231288,1.049401,1171.000000,2.822441,34.260000,-118.490000
75%,4.734400,37.000000,6.081031,1.100427,1724.750000,3.283559,37.710000,-118.010000
max,15.000100,52.000000,132.533333,34.066667,35682.000000,502.461538,41.860000,-114.310000


### 前処理

#### スケーリング

### [StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html)
データの標準化を行う。

代表的なメソッドは以下：

|メソッド|説明|
|---|---|
|fit()|標準化するための平均と分散を計算する。|
|trasform()|（事前に計算した平均と分散を使用して）標準化を行う。|
|fit_transform()|平均と分散を計算し、標準化を行う。|

In [5]:
scaler = StandardScaler()

x_train = scaler.fit_transform(x_train)
x_valid = scaler.transform(x_valid)  # x_trainの平均・分散を使用する（のはなぜ？）
x_test = scaler.transform(x_test)  # x_trainの平均・分散を使用する（のはなぜ？）

#### inputデータ分割

In [6]:
# ワイドパス、ディープパス用で分割
x_train_A, x_train_B = x_train[:, :5], x_train[:, 2:]
x_valid_A, x_valid_B = x_valid[:, :5], x_valid[:, 2:]
x_test_A, x_test_B = x_test[:, :5], x_test[:, 2:]

# モデル作成

## レイヤ構成を定義

Sequential APIでは層の順番を定義したが、
関数型APIではより複雑に層同士の接続方法を定義することが出来る。<br>

具体的には層の出力を別の層の入力に渡すことで接続方法をモデルに指示する。<br>
これがレイヤクラスのインスタンスを生成する際に入力を渡す姿が関数のように見えるため関数型APIと言われる所以である。

ちなみにインスタンス生成時は層の接続方法を指示しているだけで、データはまだ処理されていない。

In [7]:
# 初期化時にレイヤのリストを渡すことでレイヤ定義も同時に行う
input_A = keras.layers.Input(shape=[5], name='wide_input')  # ワイドパスのインプット層
input_B = keras.layers.Input(shape=[6], name='deep_input')  # ディープパスのインプット層
hidden1 = keras.layers.Dense(units=30, activation='relu')(input_B)  # input_B → hidden_1 へ
hidden2 = keras.layers.Dense(units=30, activation='relu')(hidden1)  # hidden_1 → hidden_2 へ
concat = keras.layers.concatenate([input_A, hidden2])  # input_Aとhidden_2を結合
output = keras.layers.Dense(1, name='output')(concat)  # concat →　output

model = keras.Model(inputs=[input_A, input_B], outputs=[output])

2022-07-10 21:04:27.879928: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## モデルのコンパイル

In [8]:
model.compile(loss="mse", optimizer=keras.optimizers.SGD(learning_rate=1e-3))

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 deep_input (InputLayer)        [(None, 6)]          0           []                               
                                                                                                  
 dense (Dense)                  (None, 30)           210         ['deep_input[0][0]']             
                                                                                                  
 wide_input (InputLayer)        [(None, 5)]          0           []                               
                                                                                                  
 dense_1 (Dense)                (None, 30)           930         ['dense[0][0]']                  
                                                                                              

# 学習と評価

### コールバック設定

In [9]:
import os

# ログ出力のルートディレクトリ
root_dir = os.path.join(os.curdir, 'my_logs')

# ログディレクトリ名を生成する関数
def get_run_logdir():
    import time
    run_id = time.strftime('run_%Y_%m_%d-%H_%M_%S')
    return os.path.join(root_dir, run_id)

run_logdir = get_run_logdir()

tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

In [10]:
# 早期打ち切りのコールバック関数
# 学習打ち切り時に性能が最高だった時の重みを復元するので最良モデルの保存と復元は不要
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)  # patienceで指定したエポック数学習が進まなかったときに学習を打ち切る

In [11]:
class PrintValTrainRatioCallback(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        print("\nval/train: {:.2f}".format(logs["val_loss"] / logs["loss"]))

print_valid_train_ration_cb = PrintValTrainRatioCallback()

## 学習

In [12]:
history = model.fit((x_train_A, x_train_B), y_train, epochs=20, validation_data=((x_valid_A, x_valid_B), y_valid), callbacks=[tensorboard_cb, early_stopping_cb, print_valid_train_ration_cb])

Epoch 1/20
353/363 [============================>.] - ETA: 0s - loss: 2.5674
val/train: 0.41
363/363 [==============================] - 1s 2ms/step - loss: 2.5210 - val_loss: 1.0439
Epoch 2/20
333/363 [==========================>...] - ETA: 0s - loss: 0.8225
val/train: 0.91
363/363 [==============================] - 1s 2ms/step - loss: 0.8119 - val_loss: 0.7397
Epoch 3/20
360/363 [============================>.] - ETA: 0s - loss: 0.6767
val/train: 0.97
363/363 [==============================] - 1s 1ms/step - loss: 0.6775 - val_loss: 0.6590
Epoch 4/20
344/363 [===========================>..] - ETA: 0s - loss: 0.6316
val/train: 0.99
363/363 [==============================] - 0s 1ms/step - loss: 0.6281 - val_loss: 0.6188
Epoch 5/20
339/363 [===========================>..] - ETA: 0s - loss: 0.5968
val/train: 0.99
363/363 [==============================] - 0s 1ms/step - loss: 0.5959 - val_loss: 0.5876
Epoch 6/20
325/363 [=========================>....] - ETA: 0s - loss: 0.5725
val/train: 0.

## 学習結果の可視化

In [13]:
%load_ext tensorboard
%tensorboard --logdir ./my_logs

Reusing TensorBoard on port 6006 (pid 59349), started 0:00:56 ago. (Use '!kill 59349' to kill it.)

## 学習結果の評価

In [14]:
mse_test = model.evaluate((x_test_A, x_test_B), y_test)
mse_test

162/162 [==============================] - 0s 1ms/step - loss: 0.4652


0.46522411704063416

# 学習済みモデルを使った予測

In [15]:
# サンプル用にデータサイズを限定
x_new_A, x_new_B = x_test_A[:3], x_test_B[:3]

y_pred = model.predict((x_new_A, x_new_B))
print(f'predict : {y_pred.reshape(-1)}')
print(f'correct : {y_test[:3]}')

predict : [5.7440825  2.1155465  0.77685887]
correct : [4.167 3.5   1.224]


# 複数の出力をもつニューラルネットワーク

マルチタスク分類に対応したモデルを作成する場合、出力を複数用意する必要がある。<br>
例えば同じ顔写真から表情と眼鏡を掛けているかどうかを分類するような問題がこれに当たる。

また、ユースケースとしては正則化テクニックとして使用する場合がある。<br>
例えばネットワークの下位層だけで独自に役立つ情報が学習ができるように補助出力を追加するような場合である。

今回は正則化テクニックを想定して、下位の隠れ層2層だけで独自に学習が出来るニューラルネットを作成する。

![複数出力をもつニューラルネット](./fig/multiple_outputs_nural_net.png)


モジュールインポートからデータ前処理まではワイド・アンド・ディープニューラルネットを流用する

# モデル作成

## レイヤ構成を定義

In [16]:
# 初期化時にレイヤのリストを渡すことでレイヤ定義も同時に行う
input_A = keras.layers.Input(shape=[5], name='wide_input')  # ワイドパスのインプット層
input_B = keras.layers.Input(shape=[6], name='deep_input')  # ディープパスのインプット層
hidden1 = keras.layers.Dense(units=30, activation='relu')(input_B)  # input_B → hidden_1 へ
hidden2 = keras.layers.Dense(units=30, activation='relu')(hidden1)  # hidden_1 → hidden_2 へ
concat = keras.layers.concatenate([input_A, hidden2])  # input_Aとhidden_2を結合
output = keras.layers.Dense(1, name='output')(concat)  # concat →　output
aux_output = keras.layers.Dense(1, name='aux_output')(hidden2)  # hidden_2 →　aux_output

multi_outputs_model = keras.Model(inputs=[input_A, input_B], outputs=[output, aux_output])

## モデルのコンパイル

個々の出力に対して専用の損失関数を準備する必要があるため損失関数のリストを引数に渡す。<br>
またデフォルトでは単純に個々の損失を合計してモデル全体の損失とする。重みづけする場合はloss_weightsで重みを指定する。<br>
今回の例では正則化のための補助出力よりもメイン出力の方が重要なのでメイン出力の重みを大きくする。

In [17]:
multi_outputs_model.compile(loss=["mse", "mse"], loss_weights=[0.9, 0.1], optimizer=keras.optimizers.SGD())

multi_outputs_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 deep_input (InputLayer)        [(None, 6)]          0           []                               
                                                                                                  
 dense_2 (Dense)                (None, 30)           210         ['deep_input[0][0]']             
                                                                                                  
 wide_input (InputLayer)        [(None, 5)]          0           []                               
                                                                                                  
 dense_3 (Dense)                (None, 30)           930         ['dense_2[0][0]']                
                                                                                            

# 学習と評価

## 学習

学習時もそれぞれの出力に対してラベルを指定する必要がある。<br>
今回の例では両方同じものを予測しているので同じラベルを渡す。

### コールバック設定

In [18]:
import os

# ログ出力のルートディレクトリ
root_dir = os.path.join(os.curdir, 'my_logs')

# ログディレクトリ名を生成する関数
def get_run_logdir():
    import time
    run_id = time.strftime('run_%Y_%m_%d-%H_%M_%S')
    return os.path.join(root_dir, run_id)

run_logdir = get_run_logdir()

tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

In [19]:
# 早期打ち切りのコールバック関数
# 学習打ち切り時に性能が最高だった時の重みを復元するので最良モデルの保存と復元は不要
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)  # patienceで指定したエポック数学習が進まなかったときに学習を打ち切る

In [20]:
class PrintValTrainRatioCallback(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        print("\nval/train: {:.2f}".format(logs["val_loss"] / logs["loss"]))

print_valid_train_ration_cb = PrintValTrainRatioCallback()

In [21]:
history = multi_outputs_model.fit([x_train_A, x_train_B], [y_train, y_train], epochs=20, validation_data=([x_valid_A, x_valid_B], [y_valid, y_valid]), callbacks=[tensorboard_cb, early_stopping_cb, print_valid_train_ration_cb])

Epoch 1/20
322/363 [=========================>....] - ETA: 0s - loss: 1.0985 - output_loss: 0.9180 - aux_output_loss: 2.7230
val/train: 1.45
363/363 [==============================] - 1s 2ms/step - loss: 1.0442 - output_loss: 0.8759 - aux_output_loss: 2.5588 - val_loss: 1.5171 - val_output_loss: 1.5409 - val_aux_output_loss: 1.3027
Epoch 2/20
325/363 [=========================>....] - ETA: 0s - loss: 0.6069 - output_loss: 0.5523 - aux_output_loss: 1.0986
val/train: 0.95
363/363 [==============================] - 0s 1ms/step - loss: 0.6169 - output_loss: 0.5657 - aux_output_loss: 1.0775 - val_loss: 0.5857 - val_output_loss: 0.5328 - val_aux_output_loss: 1.0621
Epoch 3/20
354/363 [============================>.] - ETA: 0s - loss: 0.5179 - output_loss: 0.4747 - aux_output_loss: 0.9064
val/train: 1.09
363/363 [==============================] - 1s 1ms/step - loss: 0.5156 - output_loss: 0.4724 - aux_output_loss: 0.9049 - val_loss: 0.5620 - val_output_loss: 0.5285 - val_aux_output_loss: 0.863

## 学習結果の可視化

In [22]:
%load_ext tensorboard
%tensorboard --logdir ./my_logs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 59349), started 0:01:06 ago. (Use '!kill 59349' to kill it.)

## 学習結果の評価

In [23]:
total_loss, main_loss, aux_loss = multi_outputs_model.evaluate((x_test_A, x_test_B), (y_test, y_test))
total_loss, main_loss, aux_loss

162/162 [==============================] - 0s 792us/step - loss: 0.3822 - output_loss: 0.3682 - aux_output_loss: 0.5089


(0.3822328448295593, 0.36815345287323, 0.508946418762207)

# 学習済みモデルを使った予測

In [24]:
# サンプル用にデータサイズを限定
x_new_A, x_new_B = x_test_A[:3], x_test_B[:3]

y_pred_main, y_pred_aux = multi_outputs_model.predict((x_new_A, x_new_B))
print(f'predict_main : {y_pred_main.reshape(-1)}')
print(f'predict_aux : {y_pred_aux.reshape(-1)}')
print(f'correct : {y_test[:3]}')

predict_main : [4.9272776 2.9948034 0.7014493]
predict_aux : [3.419609  2.6005955 0.7368641]
correct : [4.167 3.5   1.224]
